In [15]:
import numpy as np
import pandas as pd
from scipy.optimize import nnls

In [16]:
path="C:/Workplace/C++/d2ome_v2/v2/v2/bin/Debug/proportion_pxt_neh/"
time=31
pw=0.046

In [17]:
data=pd.read_csv(path+"__FAS_MOUSE.csv")
data=data[data.exp_Time==time]
data=data[data.exp_Time==time].reset_index(drop="True")
data.head()

,Peptied,charge,exp_Time,NEH,e_NEH,pxt,e_pxt,mo,m1,m2,m3,m4,m5,io,i1,i2,i3,i4,i5
0,TLLEGSGLESIINIIHSSLAEPR,3,31,47,38,0.046,0.050,0.24379,0.32816,0.23693,0.12068,0.04837,0.01617,0.034658,0.121995,0.215088,0.258925,0.217512,0.151821
1,SDEAVKPLGVK,3,31,19,17,0.046,0.050,0.51742,0.32497,0.11802,0.03140,0.00674,0.00123,0.215851,0.334139,0.251444,0.133123,0.049267,0.016175
2,FPQLDDTSFANSR,2,31,26,75,0.046,0.016,0.42316,0.34620,0.15936,0.05314,0.01419,0.00320,0.132902,0.255630,0.273362,0.172031,0.118319,0.047757
3,LLLPEDPLISGLLNSQALK,2,31,33,31,0.046,0.050,0.29874,0.34596,0.21535,0.09473,0.03282,0.00949,0.064456,0.179742,0.247627,0.241325,0.165770,0.101079
4,VLEALLPLK,2,31,14,13,0.046,0.050,0.54143,0.32084,0.10642,0.02552,0.00489,0.00079,0.277318,0.358773,0.231034,0.098342,0.028603,0.005931


In [18]:
res=[]
for pep_index in range(data.shape[0]):
    exp_0=list(data.loc[pep_index,["mo","m1","m2","m3","m4","m5"]])
    b=list(data.loc[pep_index,["io","i1","i2","i3","i4","i5"]])#exp_t
    
    A=[]
    for i in range(6):
        A.append(([0]*i)+exp_0[:6-i])
    A=np.array(A).transpose()
#     x=np.matmul( np.linalg.inv ( np.matmul(A.transpose(),A)) , np.matmul(A.transpose(),b)  )
    x=list(nnls(A,b)[0])
    
    neh= (x[3]/x[2])*3* ((1-pw)/pw) +2
    xm=sum([(index+1)*val / sum(x) for index,val in enumerate(x)])
    res.append([data.loc[pep_index,"Peptied"],data.loc[pep_index,"charge"],data.loc[pep_index,"NEH"],neh,xm, 
               data.loc[pep_index,"e_NEH"]*data.loc[pep_index,"e_pxt"]])
#     print(data.loc[pep_index,"Peptied"],x,end="\n\n")

In [19]:
res=pd.DataFrame(res)
res.columns=["Peptide","Charge","True NEH","e NEH","sum_xi","pxt_neh"]
res["dif"]=abs(res["True NEH"]- res["e NEH"])
res["test"]=100*abs(res["sum_xi"]-res["pxt_neh"])/res["pxt_neh"]
res=res.sort_values(by="dif")
res

,Peptide,Charge,True NEH,e NEH,sum_xi,pxt_neh,dif,test
6,AcVDTALENLSTLK,2,23,22.809418,2.114323,1.099999,0.190582,92.211313
4,VLEALLPLK,2,14,11.909702,1.638789,0.650000,2.090298,152.121560
10,AQVEDAFR,2,22,19.769906,2.016994,0.994000,2.230094,102.916849
0,TLLEGSGLESIINIIHSSLAEPR,3,47,49.350573,2.985189,1.899999,2.350573,57.115290
9,VQEVQQVSTNK,2,22,24.685847,2.070262,1.028999,2.685847,101.191761
1,SDEAVKPLGVK,3,19,22.922562,1.853333,0.850000,3.922562,118.039340
7,VTAIYIDPATHR,3,22,17.227032,1.949773,0.924000,4.772968,111.014368
8,QAHTMDPQLR,3,24,18.479417,1.976659,0.999999,5.520583,97.666028
11,DGAWGAFR,2,17,8.177216,1.716121,0.750000,8.822784,128.816280
3,LLLPEDPLISGLLNSQALK,2,33,43.592348,2.635216,1.549999,10.592348,70.014021
